In [ ]:
import pandas as pd
from pathlib import Path
import os

# Tutorial for handling FMU models 
The aim of this tutorial is to demonstrate how to load FMU models on python using <code>ModelicaFmuModel</code> class.

# 1. Proposed model 

In this tutorial, we load and run an FMU model created with Python beforehand.

The model is a uses a resistance-capacity approach (4R2C) of a wall installed in a test bench, with : 
- inside-outside convection/conduction transfers
- Infrared transfers with the sky and surrounding
- Short wave solar radiation transfers
- External and internal temperature as boundary conditions.

The model was built using openModelica then exported as an FMU: 

<img src="images/OM_eticsmodel.png"  height="200">


# 2. Load boundary file
First, let us load measurement data on python, which will be used as our boundary conditions:

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/study_df.csv",
    index_col=0,
    parse_dates=True
)    

In [ ]:
reference_df.head()

#  3. Instantiate ModelicaFmuModel

Now, we can also load an FMU <code>ModelicaModel</code> from <code>corrai.fmu</code>: 

Attributes:
- fmu_path: Path to the FMU file.
- output_list: List of simulation output variables.
- simulation_dir: Directory for storing simulation files.

In [ ]:
from corrai.fmu import ModelicaFmuModel

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

simu_fmu = ModelicaFmuModel(
    fmu_path=Path(TUTORIAL_DIR) / "resources/etics_v0.fmu",
    output_list= [
    "T_coat_ins.T",
     "T_ins_ins.T",
     "Tw_out.T"
],
    boundary_table_name="Boundaries"
)

The ``__repr__`` method of ``ModelicaFmuModel`` can give you valuable information on the fmu content and on the Modelica classes used in the model.

In [ ]:
simu_fmu

#  4. Run a simulation

It is time to run a simulation !

A simulation is run using ``simulate()`` method, with the following arguments:
- ``property_dict`` (optional): A dictionary containing model property key and values for the simulation.
- ``simulation_options`` (optional): A dictionary defining simulation-specific settings such as start and stop times or solver types:
    - ``startTime`` : float or pandas.Timestamp
    - ``stopTime`` : float or pandas.Timestamp
    - ``stepSize`` : float or pandas.TimeDelta
    - ``outputInterval`` : float or pandas.TimeDelta. If not provided, it will
        be set equal to ``stepSize``
    - ``solver`` : str
    - ``tolerance`` : float
    - ``fmi_type`` : {"CoSimulation", "ModelExchange"}
    - ``boundary`` : pandas.DataFrame of boundary conditions

Whether it is in ``property_dict`` or in ``simulation_options``, boundary dataframe must be specified for the simulation. Otherwise, the fmu would try to read a non existing file.
For this simulation, we ask to simulate from the first time stamp in the boundary file to the last, with a 15min timestep, but a 1h output reporting frequency.

- ``solver_duplicated_keep`` (default: "last"): Handles duplicated solver indices by selecting the desired version ("last" or "first").
- ``post_process_pipeline`` (optional): A scikit-learn pipeline to apply post-processing steps on simulation results.
- ``debug_param`` (default: False): Prints the parameter_dict if enabled.
- ``debug_logging`` (default: False): Enables detailed logging for debugging purposes.
- ``logger`` (optional): A custom logger instance for capturing logs during the simulation.


In [ ]:
property_dict = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
    'Lambda_ins.k': 0.04,
}

In [ ]:
init_res_fmu = simu_fmu.simulate(
    property_dict = property_dict,
    simulation_options={
        "startTime": reference_df.index[0],
        "stopTime": reference_df.index[-1],
        "stepSize": pd.Timedelta("15min"),
        "outputInterval": pd.Timedelta("1h"),
        "boundary": reference_df
    },
    debug_logging=False
)

We can quickly plot the results:

In [ ]:
init_res_fmu.plot()